In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import plotly.graph_objects as go
import plotly.io as pio

## Path definitions

In [2]:
DATASETS_PATH = 'data/'
MASTER_DATASET = 'MBTA_Bus_Commuter_Rail_Rapid_Transit_Reliability.csv'
MASTER_DATASET_PATH = os.path.join(DATASETS_PATH, MASTER_DATASET)

# Data Extraction

In [3]:
def load_reliability_data():
    master_df = pd.read_csv(MASTER_DATASET_PATH)
    bus_data = master_df[master_df['mode_type'] == 'Bus'].drop(columns=['mode_type'])
    bus_data['date'] = pd.to_datetime(bus_data['service_date'])
    bus_data['date'] = bus_data['date'].dt.date
    return bus_data
    

# Performances during Peak and off-Peak time

In [ ]:
def prepare_peak_data(bus_data):
    """
    Prepares data by grouping bus_data by 'peak_offpeak_ind' and converting the 'date' column to period format (monthly).
    Returns a dictionary of dataframes, with the peak/off-peak indicator as the key.
    """
    possible_peaks = bus_data['peak_offpeak_ind'].unique()
    temp_sets = {}

    for peak in possible_peaks:
        peak_data = bus_data[bus_data['peak_offpeak_ind'] == peak].copy()
        peak_data['date'] = pd.to_datetime(peak_data['date']).dt.to_period('M')
        temp_sets[peak] = peak_data

    return temp_sets

def calculate_otp_and_derivative(df):
    """
    Calculates OTP (On-Time Performance) and its derivative (dy_dx) for the given dataframe.
    Returns the updated dataframe with OTP and dy_dx columns.
    """
    df['otp'] = df['otp_numerator'] / df['otp_denominator']
    df['dy_dx'] = df['otp'].diff()
    df['dy_dx'] = df['dy_dx'].fillna(0)
    return df

def identify_sharp_points(df):
    """
    Identifies sharp points in the OTP data where the change in OTP (dy_dx) is greater than a threshold.
    Returns the sharp points as a set of dates.
    """
    threshold = df['dy_dx'].std() * 2  # Adjust threshold as needed
    sharp_dates = df[abs(df['dy_dx']) > threshold].index.astype(str)
    return set(sharp_dates)

def plot_otp_trend(temp_sets, sharp_points):
    """
    Plots the OTP trend over time and marks sharp points with red dashed lines.
    """
    # plt.figure(figsize=(24, 8))
    fig = go.Figure()
    
    for label, df_set in temp_sets.items():
        df = df_set.groupby(['date']).agg({'otp_numerator': 'sum', 'otp_denominator': 'sum'}).copy()
        df = calculate_otp_and_derivative(df)
        sharp_dates = identify_sharp_points(df)
        sharp_points.update(sharp_dates)
        
        # sns.lineplot(x=df.index.astype(str), y='otp', data=df, label=label)
        # Add a line for each group to the figure
        fig.add_trace(go.Scatter(
            x=df.index.astype(str), 
            y=df['otp'], 
            mode='lines', 
            name=f"OTP Trend of {label}"
        ))

    for point in sharp_points:
        # plt.axvline(x=point, color='red', linestyle='--', alpha=0.7)
        fig.add_vline(
            x=point,
            line=dict(color="red", dash="dash", width=2),
            name=f"Sharp Point {point}"
        )
    fig.update_layout(
        title='Month-wise OTP by Peak/Off-Peak with Sharp Points Highlighted',
        xaxis_title='Service Month',
        yaxis_title='OTP',
        xaxis_tickangle=90,
        legend_title='Peak/Off-Peak',
        plot_bgcolor='white',  # Set background to white
        paper_bgcolor='white',  # Set overall figure background to white
        template='plotly'  # Default light theme
    )

    fig.show()
    pio.write_html(fig, "./Plots/Reliability/Month_wise_OTP_Peak.html")

def plot_otp_derivative(temp_sets):
    """
    Plots the derivative of OTP (change rate of OTP) over time.
    """
    fig = go.Figure()

    for label, df_set in temp_sets.items():
        df = df_set.groupby(['date']).agg({'otp_numerator': 'sum', 'otp_denominator': 'sum'}).copy()
        df = calculate_otp_and_derivative(df)

        # Add a line for each group to the figure
        fig.add_trace(go.Scatter(
            x=df.index.astype(str), 
            y=df['dy_dx'], 
            mode='lines', 
            name=f"Derivative of {label}"
        ))

    # Update layout for better presentation
    fig.update_layout(
        title='Month-wise OTP Change Rate (Derivative)',
        xaxis_title='Service Month',
        yaxis_title='Change in OTP',
        xaxis_tickangle=90,
        legend_title='Peak/Off-Peak',
    )
    fig.show()
    pio.write_html(fig, "./Plots/Reliability/Month_wise_OTP_Change_Rate.html")
    # pio.write_image(fig, "./Plots/Reliability/Month_wise_OTP_Change_Rate.jpg", format="jpg", width=500, height=300, scale=1)

# Main execution
bus_data = load_reliability_data() 
temp_sets = prepare_peak_data(bus_data)
sharp_points = set()

In [8]:
plot_otp_trend(temp_sets, sharp_points)

In [9]:
plot_otp_derivative(temp_sets)

# On time performance for each bus over the years

In [11]:
def prepare_data(bus_data):
    """
    Prepares the bus data by converting the service date to periods and 
    aggregating OTP values by route and date.
    
    Args:
    bus_data (DataFrame): The raw bus data containing service_date, otp_numerator, and otp_denominator.

    Returns:
    DataFrame: Processed data grouped by route and date, with OTP values calculated.
    """
    route_level_data = bus_data.copy()

    # Ensure the route short name is set properly
    route_level_data.loc[:, 'gtfs_route_short_name'] = route_level_data['gtfs_route_short_name'].where(
        route_level_data['gtfs_route_short_name'].notna(), route_level_data['gtfs_route_id']
    )

    # Convert service_date to datetime and then to 6-month periods
    route_level_data.loc[:, 'date'] = pd.to_datetime(route_level_data['service_date']).dt.to_period('Y')
    route_level_data = route_level_data.groupby(['gtfs_route_short_name', 'date']).agg({'otp_numerator': 'sum', 'otp_denominator': 'sum'}).reset_index()
    
    return route_level_data

def plot_otp(route_level_data, important_routes, color_palette):
    """
    Plots OTP data for the specified important bus routes using Plotly.
    
    Args:
    route_level_data (DataFrame): The processed bus data with OTP values.
    important_routes (list): List of bus routes to plot.
    color_palette (list): List of colors for the plot.
    
    Returns:
    None: Displays the plot.
    """
    fig = go.Figure()

    for idx, bus_name in enumerate(important_routes):
        single_bus_data = route_level_data[route_level_data['gtfs_route_short_name'] == bus_name]
        single_bus_data['otp'] = single_bus_data['otp_numerator'] / single_bus_data['otp_denominator']

        # Add a line for each bus route
        fig.add_trace(go.Scatter(
            x=single_bus_data['date'].astype(str),
            y=single_bus_data['otp'],
            mode='lines',
            name=bus_name,
            line=dict(color=color_palette[idx])
        ))

    # Customize the layout
    fig.update_layout(
        title='OTP Bus wise, every 6 months',
        xaxis_title='Service 6-Month Period',
        yaxis_title='OTP',
        legend_title='Bus Routes',
        xaxis_tickangle=90
    )

    fig.show()
    pio.write_html(fig, "./Plots/Reliability/OTP_performance.html")

route_level_data = prepare_data(bus_data)
important_routes = ['22', '29', '15', '45', '44', '42', '17', '23', '31', '26', '111', '24', '33', '14']
color_palette = ["Maroon", "Grey", "Olive", "Teal", "Navy", "Black", "Red", "Orange", "Yellow", "Green", "Cyan", "Blue", "Purple", "Magenta"]
plot_otp(route_level_data, important_routes, color_palette)

C:\Users\astha\AppData\Local\Temp\ipykernel_44804\1075755586.py:20: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.

C:\Users\astha\AppData\Local\Temp\ipykernel_44804\1075755586.py:41: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\astha\AppData\Local\Temp\ipykernel_44804\1075755586.py:41: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\astha\AppData\Local\Temp\ipykernel_44804\1075755586.py:41: SettingWithCopyWarning:


A value is trying to be set 